#Chicago Marathon data scraper

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import time
import datetime
import json
import string
import os 
import re

#Small helpers

## Converts 01:30:00 -> 5400
## Discards decimals ...
def timestring_to_sec(ts):
    if pd.notnull(ts):
        return sum(int(x) * 60 ** i for i,x in enumerate(reversed(ts.split(".")[0].split(":"))))
    else:
        return ts

## Converts 01:30:00 -> 90

def timestring_to_min(ts):
    if pd.notnull(ts):
        return timestring_to_sec(ts)/60.
    else:
        return ts

## Converts 5400 -> 01:30:00 
def sec_to_timestring (seconds): 
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    string = "%d:%02d:%02d" % (h, m, s)
    return string

## 90 -> 01:30:00 
min_to_timestring = lambda min: sec_to_timestring(min*60)

First we download the tables of 1000 results each, so we can gather links to individual runner data 

In [8]:
# Basic scraper for Chicago Marathon results

# URL format


year = "2015"
url_start = "http://results.chicagomarathon.com/"+ year + "/?page=" # swap to 2015 for 2015
url_mid = "&event=MAR&lang=EN_CAP&num_results=1000&pid=list&search[sex]="

# Needs to end in M for men and W for women

# We know from manual browsing that there are 21 pages for men and 18 for women in 2015,
# 23 pages for men and 19 for women in 2014.

pages_text_m = []
pages_text_w = []
count_m = 23
count_w = 18

def get_pages(pages_text_list, count, gender):
    for p in range(1, count + 1):
        # so we know the progress
        print p,
        pages_text_list.append(bs(requests.get(url_start + str(p) + url_mid + gender).text,
                                  "html"))
        time.sleep(1)
        
#get_pages(pages_text_m, count_m, "M")
get_pages(pages_text_w, count_w, "W")


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18


In [3]:
#Scrap split times for individual runner
def get_splits(url):
    time.sleep(.01)
    soup = bs(requests.get(url).text,"html")
    tables = soup.findAll("table", { "class" : "list-table names" })
 
    runner = pd.read_html(tables[0].prettify(), index_col=0)[0][1]
    splits = pd.read_html(tables[4].prettify(), index_col=0)[0]['Time']
    return pd.concat([runner,splits], axis=0)


url = "http://results.chicagomarathon.com/2015/?content=detail&fpid=list&pid=list&idp=999999107FA30900001756C8&lang=EN_CAP&event=MAR&lang=EN_CAP&num_results=1000&search%5Bsex%5D=M&search_event=MAR"
get_splits(url)

Name (CTZ)     Chumba, Dickson (KEN)
Age Group                      25-29
Bib Number                         3
Age                               28
City, State                      NaN
05K                         00:15:31
10K                         00:30:46
15K                         00:46:00
20K                         01:01:46
HALF                        01:05:13
25K                         01:17:24
30K                         01:33:16
35K                         01:47:52
40K                         02:02:43
Finish                      02:09:25
dtype: object

##Actually do the scraping -- may take up to four hours to run!

In [6]:
def get_split_table(soup):
    print ".",
    base = "http://results.chicagomarathon.com/" + year + "/"
    links = soup.findAll("table", { "class" : "list-table" })[0].findAll('a',href=re.compile('^\?content=detail')) 
    results = [get_splits((base + l['href'])) for l in links]
    return pd.concat(results,axis=1).transpose()

print "Scraping men"
#%time men = [get_split_table(p) for p in pages_text_m]
men = []
for p in pages_text_m:
    men.append(get_split_table(p))

print "Scraping women"
#%time women = [get_split_table(p) for p in pages_text_w]


 Scraping men
. . . . . . . . . . . . . . . . . . . . . .

ValueError: All objects passed were None

In [7]:
len(men)

21

In [11]:
women = []
for p in pages_text_w:
    women.append(get_split_table(p))



. . . . . . . . . . . . . . . . . .


In [12]:
len(women)

18

In [13]:
pd.concat(men,axis=1)
men_df = pd.concat(men,axis=0)
men_df['Gender1F2M'] = 2
men_df.head()

,05K,10K,10K *,15K,15K *,20K,20K *,25K,25K *,30K,30K *,35K,35K *,40K,40K *,Age,Age Group,Bib Number,"City, State",Finish,HALF,HALF *,Name (CTZ),Gender1F2M
0,00:15:31,00:30:46,NaN,00:46:00,NaN,01:01:46,NaN,01:17:24,NaN,01:33:16,NaN,01:47:52,NaN,02:02:43,NaN,28,25-29,3,NaN,02:09:25,01:05:13,NaN,"Chumba, Dickson (KEN)",2
1,00:15:31,00:30:46,NaN,00:46:01,NaN,01:01:45,NaN,01:17:24,NaN,01:33:14,NaN,01:47:52,NaN,02:03:02,NaN,28,25-29,2,NaN,02:09:50,01:05:13,NaN,"Kitwara, Sammy (KEN)",2
2,00:15:32,00:30:47,NaN,00:46:03,NaN,01:01:47,NaN,01:17:26,NaN,01:33:16,NaN,01:48:08,NaN,02:03:24,NaN,27,25-29,10,NaN,02:10:06,01:05:13,NaN,"Ndungu, Sammy (KEN)",2
3,00:15:33,00:30:46,NaN,00:46:01,NaN,01:01:46,NaN,01:17:24,NaN,01:33:15,NaN,01:48:07,NaN,02:03:11,NaN,27,25-29,6,NaN,02:10:07,01:05:13,NaN,"Gebru, Girmay Birhanu (ETH)",2
4,00:15:33,00:30:48,NaN,00:46:03,NaN,01:01:48,NaN,01:17:25,NaN,01:33:15,NaN,01:48:10,NaN,02:03:36,NaN,25,25-29,18,NaN,02:10:24,01:05:14,NaN,"Puskedra, Luke (USA)",2


In [15]:
women_df = pd.concat(women,axis=0)
women_df['Gender1F2M'] = 1
women_df.head()

,05K,10K,10K *,15K,15K *,20K,20K *,25K,25K *,30K,30K *,35K,35K *,40K,40K *,Age,Age Group,Bib Number,"City, State",Finish,HALF,HALF *,Name (CTZ),Gender1F2M
0,00:16:34,00:33:06,NaN,00:49:40,NaN,01:06:40,NaN,01:24:00,NaN,01:41:19,NaN,01:58:44,NaN,02:15:58,NaN,28,25-29,101,NaN,02:23:33,01:10:28,NaN,"Kiplagat, Florence (KEN)",1
1,00:16:35,00:33:07,NaN,00:49:42,NaN,01:06:41,NaN,01:24:00,NaN,01:41:19,NaN,01:58:44,NaN,02:16:04,NaN,25,25-29,106,NaN,02:23:43,01:10:28,NaN,"Melese, Yebrgual (ETH)",1
2,00:16:34,00:33:06,NaN,00:49:41,NaN,01:06:40,NaN,01:24:00,NaN,01:41:19,NaN,01:58:44,NaN,02:16:18,NaN,22,20-24,102,NaN,02:24:24,01:10:28,NaN,"Dibaba, Birhane (ETH)",1
3,00:16:34,00:33:05,NaN,00:49:42,NaN,01:06:39,NaN,01:24:01,NaN,01:41:20,NaN,01:58:45,NaN,02:16:32,NaN,33,30-34,108,NaN,02:24:25,01:10:27,NaN,"Fukushi, Kayoko (JPN)",1
4,00:16:35,00:33:08,NaN,00:49:42,NaN,01:06:40,NaN,01:24:01,NaN,01:41:19,NaN,01:58:44,NaN,02:16:27,NaN,31,30-34,105,NaN,02:24:40,01:10:28,NaN,"Seboka, Mulu (ETH)",1


In [16]:
Chicago_df = pd.concat([women_df,men_df],axis=0)
Chicago_df['Year'] = year
Chicago_df.tail()

,05K,10K,10K *,15K,15K *,20K,20K *,25K,25K *,30K,30K *,35K,35K *,40K,40K *,Age,Age Group,Bib Number,"City, State",Finish,HALF,HALF *,Name (CTZ),Gender1F2M,Year
182,00:34:05,01:13:32,NaN,01:56:27,NaN,02:50:05,NaN,03:47:59,NaN,04:43:22,NaN,05:45:23,NaN,06:44:49,NaN,24,20-24,19726,Chicago,07:08:52,03:07:24,NaN,"Delgado, Alfredo (USA)",2,2015
183,01:38:47,02:19:06,NaN,02:56:49,NaN,03:37:58,NaN,04:21:04,NaN,05:06:05,NaN,05:54:27,NaN,06:46:14,NaN,44,40-44,55191,Frankfort,07:09:38,03:47:00,NaN,"Gupta, Anand (IND)",2,2015
184,00:39:49,01:20:42,NaN,02:07:36,NaN,02:54:34,NaN,03:50:44,NaN,04:43:06,NaN,05:45:12,NaN,06:49:04,NaN,54,50-54,57995,Chicago,07:13:38,03:05:35,NaN,"Becker, Robert (USA)",2,2015
185,00:38:57,01:20:26,NaN,02:03:27,NaN,02:53:05,NaN,03:44:30,NaN,04:46:25,NaN,05:55:44,NaN,06:58:54,NaN,69,65-69,16762,Chicago,07:24:21,03:03:40,NaN,"Aron, Chuck (USA)",2,2015
186,00:48:54,01:37:46,NaN,02:27:27,NaN,03:19:28,NaN,04:14:30,NaN,05:09:55,NaN,06:05:00,NaN,07:02:12,NaN,74,70-74,7292,Chicago,07:26:27,03:31:18,NaN,"Moon, Larry (USA)",2,2015


In [17]:
#Save file to disk
fn = "Chicago" + year + "Original.csv"
Chicago_df.to_csv(fn, header=True, index=False, encoding='utf-8')

#Chicago_df2 = pd.read_csv(fn, header=True, encoding='utf-8')

Function below does the following:
-	Converts dashes to nans
-	Renames split headers in accordance with our data format
-	 Converts from split aggregate times to split differences


In [19]:
dash_to_nan = lambda s: np.nan if s == '-' else s
mappings = {'05K':'K0-5','10K':'K5-10','15K':'K10-15', '20K':'K15-20', '25K':'K20-25', '30K':'K25-30','35K':'K30-35','40K':'K35-40','Finish':'K40-Fin','HALF':'HalfMar'}

def reformat_df(df):
    global year
    df_c = df.copy()
    
    for k,v in mappings.items():
        df_c[v] = df_c[k].apply(dash_to_nan)
        df_c[v] = df_c[v].apply(timestring_to_min)

    df_c.index = df_c['Bib Number']
    df_c.index.names = ['BibNum']
    
    df_meta = df_c[['Year','Age','Gender1F2M']].copy()
    df_meta['StartHr'] = np.nan
    df_meta['StartMin'] = np.nan
    
    df_sub = df_c[['K40-Fin','K35-40','K30-35','K25-30','K20-25','K15-20','K10-15','K5-10','K0-5']].copy()
    df_sub['K0'] = 0
    df_filled = df_sub.interpolate(axis=1)
    
    df_diff = pd.DataFrame()
    for i,c in enumerate(df_filled.columns):
        maxlength = (len(df_filled.columns) - 1) 
        if (i < maxlength):
            df_diff[c] = df_filled[c] - df_filled[(df_filled.columns[i+1])]
        
    
    splits= df_diff[df_diff.columns[::-1]]
    fdf = pd.concat([df_meta,splits, df_c['HalfMar']],axis =1)
    fdf['Age2014'] = fdf.Age.apply(int) - (int(year)-2014)
    return fdf
     
diffed = reformat_df(Chicago_df)                   
diffed.head()

,Year,Age,Gender1F2M,StartHr,StartMin,K0-5,K5-10,K10-15,K15-20,K20-25,K25-30,K30-35,K35-40,K40-Fin,HalfMar,Age2014
BibNum,,,,,,,,,,,,,,,,
101,2015,28,1,NaN,NaN,16.566667,16.533333,16.566667,17.000000,17.333333,17.316667,17.416667,17.233333,7.583333,70.466667,27
106,2015,25,1,NaN,NaN,16.583333,16.533333,16.583333,16.983333,17.316667,17.316667,17.416667,17.333333,7.650000,70.466667,24
102,2015,22,1,NaN,NaN,16.566667,16.533333,16.583333,16.983333,17.333333,17.316667,17.416667,17.566667,8.100000,70.466667,21
108,2015,33,1,NaN,NaN,16.566667,16.516667,16.616667,16.950000,17.366667,17.316667,17.416667,17.783333,7.883333,70.450000,32
105,2015,31,1,NaN,NaN,16.583333,16.550000,16.566667,16.966667,17.350000,17.300000,17.416667,17.716667,8.216667,70.466667,30


In [20]:
#Save to disk
fn = "Chicago" + year + "Formated.csv"
diffed.to_csv(fn, header=True, index=True, encoding='utf-8')